# Tutorial 2: How does evaluation of local matrix and vector work

Go deep into the evaluation of weak form functions

Activate and instantiate packages (See tutorial_1)

In [3]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using Gridap

  Activating project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`


Define geometry, model, weak form (see tutorial_1)

In [4]:
L = 2.0
n = 2
domain = (0,L,0,L)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)
reffe = ReferenceFE(lagrangian,Float64,1)
V = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V,x->x[1])
Ω = Triangulation(model)
dΩ = Measure(Ω,2)
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
l(v) = ∫( f*v )dΩ


l (generic function with 1 method)

Get basis functions

In [5]:
u_basis = get_trial_fe_basis(U) # basis functions of the trial space
v_basis = get_fe_basis(V)   # basis functions of the test space

SingleFieldFEBasis():
 num_cells: 4
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 13853805979276197901

Get matvec contributions (evaluate local matrix and vector)

In [6]:
matcontrib = a(u_basis,v_basis)
veccontrib = l(v_basis)

DomainContribution()

In [7]:
first(matcontrib.dict)

BodyFittedTriangulation() => Fill([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], 4)

Get solution vector

In [8]:
first(veccontrib.dict)

BodyFittedTriangulation() => [[0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999]]

Go deep into integrate function

In [14]:
using Gridap.CellData: DomainContribution, add_contribution!
object  =  ∇(u_basis)⋅∇(v_basis)
cell_mats = integrate(object,dΩ.quad)
matcontrib2 = DomainContribution()
add_contribution!(matcontrib2,dΩ.quad.trian,cell_mats)

DomainContribution()

a(u,v) is the same as integrate(object,quad)

In [22]:
using Gridap.CellData: get_domains
get_contribution(matcontrib, first(get_domains(matcontrib))) == get_contribution(matcontrib2, first(get_domains(matcontrib2)))

true

Let's explore integrate function:

get quadrature points

In [30]:
quad = dΩ.quad
x = get_cell_points(quad)

CellPoint()

evaluate object at quadrature points

In [31]:
bx = object(x)

4-element Fill{Array{Float64, 3}}, with entries equal to [1.2440169358562925 -0.4553418012614796 -0.4553418012614796 -0.33333333333333326; 0.6666666666666667 -0.45534180126147955 0.12200846792814618 -0.3333333333333333; 0.6666666666666667 0.12200846792814618 -0.45534180126147955 -0.3333333333333333; 0.08931639747704091 0.12200846792814624 0.12200846792814624 -0.33333333333333337;;; -0.4553418012614796 0.6666666666666666 -0.33333333333333326 0.1220084679281462; -0.45534180126147955 1.2440169358562925 -0.3333333333333333 -0.4553418012614796; 0.12200846792814618 0.0893163974770409 -0.3333333333333333 0.12200846792814625; 0.12200846792814624 0.6666666666666667 -0.33333333333333337 -0.45534180126147955;;; -0.4553418012614796 -0.33333333333333326 0.6666666666666666 0.1220084679281462; 0.12200846792814618 -0.3333333333333333 0.0893163974770409 0.12200846792814625; -0.45534180126147955 -0.3333333333333333 1.2440169358562925 -0.4553418012614796; 0.12200846792814624 -0.33333333333333337 0.666666

In [33]:
quad.data_domain_style

ReferenceDomain()

In [34]:
quad.integration_domain_style


PhysicalDomain()

In [37]:
@show cell_map = get_cell_map(quad.trian)
@show cell_Jt = lazy_map(∇,cell_map)
@show cell_Jtx = lazy_map(evaluate,cell_Jt,quad.cell_point);

cell_map = get_cell_map(quad.trian) = Gridap.Fields.AffineField{2, 2, Float64, 4}[AffineField() AffineField(); AffineField() AffineField()]
cell_Jt = lazy_map(∇, cell_map) = Fill(ConstantField(), 2, 2)
cell_Jtx = lazy_map(evaluate, cell_Jt, quad.cell_point) = Fill(TensorValue{2, 2, Float64, 4}[(1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0)], 4)


In [39]:
using Gridap.Fields: IntegrationMap
lazy_map(IntegrationMap(),bx,quad.cell_weight,cell_Jtx)

4-element Fill{Matrix{Float64}}, with entries equal to [0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663]